In [1]:
import pandas as pd
import numpy as np 
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy import MetaData
import sqlite3
import datetime

In [2]:
engine = create_engine('sqlite:///northwind.db') #Creacion de la conexion de la base de datos

In [3]:
frameAlbums = pd.read_sql_query("""SELECT  * from Employees""", con=engine.connect())
frameAlbums

OperationalError: (sqlite3.OperationalError) no such table: Employees
[SQL: SELECT  * from Employees]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
metadata = MetaData()
metadata.create_all(engine)
inspector = inspect(engine)
inspector.get_columns('Customers')

In [ ]:
df_customer = pd.read_sql_query(
"""
SELECT 
    CompanyName,  
    ContactName,  
    ContactTitle, 
    Address,      
    City,         
    Region,     
    ifnull(PostalCode ,'No Asigando')as PostalCode,
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country, 
    Phone,
    Fax            
FROM Customers;
"""
,con=engine.connect())
df_customer.head(8)


In [ ]:
df_employe = pd.read_sql_query(
"""
SELECT 
    LastName,
    FirstName,       
    Title,         
    TitleOfCourtesy, 
    BirthDate  ,    
    HireDate ,      
    Address,        
    City  ,         
    Region,        
    PostalCode,      
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country,        
    HomePhone,      
    Extension,       
    ifnull(Photo ,'No Asigando')as Photo,             
    Notes,          
    ifnull(ReportsTo ,'No Asigando')as ReportsTo,          
    PhotoPath             
FROM Employees;
"""
,con=engine.connect())
df_employe.head(4)

In [ ]:
df_location = pd.read_sql_query(
"""
SELECT  
    ShipName,
    ShipAddress,   
    ShipCity  ,    
    ShipRegion ,    
    ifnull(ShipPostalCode ,'No Asigando')as ShipPostalCode,     
    replace(replace(ShipCountry,"UK","United Kingdom"),"USA","The United States of America") as ShipCountry              
                   
FROM Orders;
"""
,con=engine.connect())
df_location.head(4)

In [ ]:
df_produc = pd.read_sql_query(
"""
SELECT 
   
    ProductName,
    SupplierId,
    CategoryId,                              
    QuantityPerUnit, 
    UnitPrice,      
    UnitsInStock,    
    UnitsOnOrder,   
    ReorderLevel,   
    Discontinued           
FROM Products;
"""
,con=engine.connect())
df_produc.head(4)

In [ ]:
df_shipper = pd.read_sql_query(
"""
SELECT Id As shipperId, 
    CompanyName,
    Phone           
FROM Shippers;
"""
,con=engine.connect())
df_shipper.head(4)

In [ ]:
df_supplier = pd.read_sql_query(
"""
SELECT 
                
    CompanyName ,
    ContactName  ,
    ContactTitle ,
    Address      ,
    City        ,
    Region      ,
    PostalCode  ,
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country ,           
    ifnull(Fax ,'No Asigando')as Fax,
    ifnull(HomePage ,'No Asigando')as HomePage,         
    Phone    
    FROM Suppliers;
"""
,con=engine.connect())
df_supplier.head(4)

In [ ]:
fecha_inicio='2009-01-01'
fecha_fin='2030-12-31'
dataFrameDate = pd.DataFrame({'fecha': pd.date_range(fecha_inicio, fecha_fin)})
dataFrameDate['año'] = dataFrameDate.fecha.dt.year
dataFrameDate['mes'] = dataFrameDate.fecha.dt.month
dataFrameDate['dia'] = dataFrameDate.fecha.dt.day
dataFrameDate['nombre_dia'] = dataFrameDate.fecha.dt.day_name()
dataFrameDate['dia_semana'] = dataFrameDate.fecha.dt.dayofweek
dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear
dataFrameDate['trimestre'] = dataFrameDate.fecha.dt.quarter
dataFrameDate = dataFrameDate[['fecha', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']] 
dataFrameDate

In [ ]:
TablaHechos = pd.read_sql_query(
"""   
SELECT OrderDetail.Id AS FactId ,Customer.Id AS CustomerId, Employee.Id AS EmployeeId, Orders.Id AS LocationId, Orders.OrderDate,Product.Id AS ProductId , Shipper.Id AS ShipperId, Supplier.Id AS SupplierId, OrderDetail.UnitPrice
FROM OrderDetail
 INNER JOIN Orders  ON Orders.Id = OrderDetail.OrderId 
INNER JOIN  Customers ON Customer.Id = Orders.CustomerId 
INNER JOIN Employees  ON Employee.Id = Orders.EmployeeId
INNER JOIN Products  ON Product.Id = OrderDetail.ProductId
 INNER JOIN Shippers  ON Shipper.Id = Orders.ShipVia 
 INNER JOIN Suppliers ON Supplier.Id = Product.SupplierId ;
"""
,con=engine.connect())
TablaHechos.head(4)

In [ ]:
con2 = create_engine('sqlite:///DW_Northwind_Estrella.sqlite')
con2

In [ ]:
df_customer.to_sql("dim_customers",con2, if_exists='append',index = False, )

In [ ]:
def updateData(name_table,data_db,name_columns):
	print(name_columns)
	aux_values='?'
	aux_data=[]
	aux_columns=name_columns[0]
	aux_data.append(list(data_db[name_columns[0]]))
	entities=[]
	aux_entities=[]
	for i in range(len(name_columns)-1):
		aux_values+=',?'
		aux_columns+=','+name_columns[i+1]
		aux_data.append(list(data_db[name_columns[i+1]]))
	for i in range(len(aux_data[0])):
		for j in range(len(aux_data)):
			aux_entities.append(aux_data[j][i])
		entities.append(aux_entities)
		aux_entities=[]
	with con2.connect() as con:
		for i in entities:
			con.execute('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')', i)
	con.close()

In [ ]:
updateData('dim_customer',df_customer,list(df_customer.columns.values))
updateData('dim_location',df_location,list(df_location.columns.values))
updateData('dim_employee',df_employe ,list(df_employe.columns.values))
updateData('dim_product',df_produc,list(df_produc.columns.values))
updateData('dim_shipper',df_shipper,list(df_shipper.columns.values))
updateData('dim_supplier',df_supplier,list(df_supplier.columns.values))
dataFrameDate.to_sql(name='dim_time', con=engine.connect(), if_exists='append',index=False)
print()
updateData('factsales_order',TablaHechos,list(TablaHechos.columns.values))